**Name: MD Fahimul Islam**

**Reg No: 2017331026**

# GAME PLAYING: Tic Tac Toe | Connect Four | N-Queen | Graph Coloring | 8 Puzzle**

In [ ]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools 
cache = functools.lru_cache(10**6)

class Game is similar to a problem in Adhversarial search problem..This game has :

1) actions (Return a collection of the allowable moves from this state)

2) result (Return the state that results from making a move from a state)

3) is_terminal (Return True if this is a final state for the game)

4) utility function (Return the value of this final state to player) functions.


In [ ]:
class Game:
  def actions(self, state):
    raise NotImplementedError

  def result(self, state, move):
    raise NotImplementedError

  def is_terminal(self, state):
    return not self.actions(state)
  
  def utility(self, state, player):
    raise NotImplementedError
        

def play_game(game, strategies: dict, verbose=False):
  # Play a turn-taking game. 
  # strategies is a {player_name: function} dict, where function(state, game) is used to get the player's move."""
  state = game.initial
  while not game.is_terminal(state):
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state, move)
    if verbose: 
        print('Player', player, 'move:', move)
        print("State: ")
        print(state)
  return state

**MINIMAX:**

**The optimal strategy can be determined from the minimax value of each node, which we write as MINIMAX(n). The minimax value of a node is the utility value (for MAX) of being in the corresponding state, assuming that both players play optimally from that state to the end of the game. Obviously, the minimax value of a terminal state is just its utility. Furthermore, given a choice, MAX prefers to move to a state of maximum value, whereas MIN prefers a state of minimum value**



In [ ]:

def minimax_search(game, state):
  player = state.to_move  #state.to move outputs that player who's turn is now

  def max_value(state):
    if game.is_terminal(state):
      return game.utility(state, player), None
    value, move = -infinity, None
    for a in game.actions(state):
      v2, _ = min_value(game.result(state, a))
      if v2 > value:
        value, move = v2, a
    return value, move

  def min_value(state):
    if game.is_terminal(state):
      return game.utility(state, player), None
    value, move = +infinity, None
    for a in game.actions(state):
      v2, _ = max_value(game.result(state, a))
      if v2 < value:
          value, move = v2, a
    return value, move

  return max_value(state)

infinity = math.inf



**ALPHA–BETA PRUNING:**

**The problem with minimax search is that the number of game states it has to examine is exponential in the depth of the tree. When we apply ALPHA–BETA PRUNING  to a standard minimax tree, it returns the same move as minimax would, but prunes away branches that cannot possibly influence the final decision**

In [ ]:
def alphabeta_search(game, state):
  player = state.to_move

  def max_value(state, alpha, beta):
    if game.is_terminal(state):
      return game.utility(state, player), None
    value, move = -infinity, None
    for a in game.actions(state):
      v2, _ = min_value(game.result(state, a), alpha, beta)
      if v2 > value:
          value, move = v2, a
          alpha = max(alpha, value)
      if value >= beta:
          return value, move
    return value, move

  def min_value(state, alpha, beta):
    if game.is_terminal(state):
      return game.utility(state, player), None
    value, move = +infinity, None
    for a in game.actions(state):
      v2, _ = max_value(game.result(state, a), alpha, beta)
      if v2 < value:
        value, move = v2, a
        beta = min(beta, value)
      if value <= alpha:
        return value, move
    return value, move

  return max_value(state, -infinity, +infinity)

**Tic Tac Toe:**

In [ ]:

class TicTacToe(Game):

  def __init__(self, height=3, width=3, k=3):
    self.k = k # k in a row
    self.squares = {(x, y) for x in range(width) for y in range(height)}
    self.initial = Board(height=height, width=width, to_move='X', utility=0)

  def actions(self, board):
    # Legal moves
    return self.squares - set(board)

  def result(self, board, square):
    """Place a marker for current player on square."""
    player = board.to_move
    board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
    win = k_in_row(board, player, square, self.k)
    board.utility = (0 if not win else +1 if player == 'X' else -1)
    return board

  def utility(self, board, player):
    """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
    return board.utility if player == 'X' else -board.utility

  def is_terminal(self, board):
    """A board is a terminal state if it is won or there are no empty squares."""
    return board.utility != 0 or len(self.squares) == len(board)

  def display(self, board): print(board)     


def k_in_row(board, player, square, k):
  """True if player has k pieces in a line through square."""
  def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
  return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
              for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

In [ ]:
class Board(defaultdict):
  empty = '.'
  off = '#'
  
  def __init__(self, width=8, height=8, to_move=None, **kwds):
    self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
      
  def new(self, changes: dict, **kwds) -> 'Board':
    board = Board(width=self.width, height=self.height, **kwds)
    board.update(self)
    board.update(changes)
    return board

  def __missing__(self, loc):
    x, y = loc
    if 0 <= x < self.width and 0 <= y < self.height:
        return self.empty
    else:
        return self.off
          
  def __hash__(self): 
    return hash(tuple(sorted(self.items()))) + hash(self.to_move)
  
  def __repr__(self):
      def row(y): return ' '.join(self[x, y] for x in range(self.width))
      return '\n'.join(map(row, range(self.height))) +  '\n'

In [ ]:
def random_player(game, state): return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

**Optimal Player(X) VS Random Player(0):**

**X is MAX player who is a alphabeta player who always takes optimal move.0 is MIN player who is a random player who moves randomly.**


**The alpha-beta player will never lose (means utility value of X is 1) , but sometimes the random player can achieve a draw (means both's utility value is 0.**

***So Utility of X is:***

In [ ]:
a = play_game(TicTacToe(), dict(X=player(alphabeta_search), O=random_player), verbose=True)
print("X is MAX player who is a alphabeta player who always takes optimal move")
print("0 is MIN player who is a random player who moves randomly")
print("Utility Value of player X: ",a.utility)

Player X move: (0, 1)
State: 
. . .
X . .
. . .

Player O move: (1, 0)
State: 
. O .
X . .
. . .

Player X move: (0, 0)
State: 
X O .
X . .
. . .

Player O move: (2, 0)
State: 
X O O
X . .
. . .

Player X move: (1, 2)
State: 
X O O
X . .
. X .

Player O move: (0, 2)
State: 
X O O
X . .
O X .

Player X move: (1, 1)
State: 
X O O
X X .
O X .

Player O move: (2, 2)
State: 
X O O
X X .
O X O

Player X move: (2, 1)
State: 
X O O
X X X
O X O

X is MAX player who is a alphabeta player who always takes optimal move
0 is MIN player who is a random player who moves randomly
Utility Value of player X:  1


**Optimal Player(X) VS Optimal Player(0):**

**X is a MAX player who is a alphabeta player who always takes optimal move.0 is MIN player who is a minimax player who always takes optimal move.**


**When two optimal (alpha-beta or minimax) players compete, it will always be a draw:**

***So Utility of X is:***

In [ ]:
a = play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True)
print("X is MAX player who is a alphabeta player who always takes optimal move")
print("0 is MIN player who is a minimax player who always takes optimal move")
print("Utility Value of player X: ",a.utility)

Player X move: (0, 1)
State: 
. . .
X . .
. . .

Player O move: (0, 0)
State: 
O . .
X . .
. . .

Player X move: (2, 0)
State: 
O . X
X . .
. . .

Player O move: (2, 1)
State: 
O . X
X . O
. . .

Player X move: (1, 2)
State: 
O . X
X . O
. X .

Player O move: (0, 2)
State: 
O . X
X . O
O X .

Player X move: (1, 0)
State: 
O X X
X . O
O X .

Player O move: (1, 1)
State: 
O X X
X O O
O X .

Player X move: (2, 2)
State: 
O X X
X O O
O X X

X is MAX player who is a alphabeta player who always takes optimal move
0 is MIN player who is a minimax player who always takes optimal move
Utility Value of player X:  0


**Connect Four:**

**Connect Four is a variant of tic-tac-toe which is played on a larger (7 x 6) board, and with the restriction that in any column we can only play in the lowest empty square in the column.**

In [ ]:
class ConnectFour(TicTacToe):
    
  def __init__(self): super().__init__(width=7, height=6, k=4)

  def actions(self, board):
      """In each column you can play only the lowest empty square in the column."""
      return {(x, y) for (x, y) in self.squares - set(board)
              if y == board.height - 1 or (x, y + 1) in board}

In [ ]:
a = play_game(ConnectFour(), dict(X=random_player, O=random_player), verbose=True)
print("Utility Value of player X: ",a.utility)

Player X move: (6, 5)
State: 
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . X

Player O move: (2, 5)
State: 
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . X

Player X move: (4, 5)
State: 
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . X . X

Player O move: (4, 4)
State: 
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
. . O . X . X

Player X move: (0, 5)
State: 
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
X . O . X . X

Player O move: (4, 3)
State: 
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
. . . . O . .
X . O . X . X

Player X move: (4, 2)
State: 
. . . . . . .
. . . . . . .
. . . . X . .
. . . . O . .
. . . . O . .
X . O . X . X

Player O move: (2, 4)
State: 
. . . . . . .
. . . . . . .
. . . . X . .
. . . . O . .
. . O . O . .
X . O . X . X

Player X move: (1, 5)
State: 
. . . . . . .
. . . . . . .
. . . . X . .
. . . . 

**N-Queen:**

In [ ]:
# N = Number of queens

N = 8

#NxN matrix with all elements 0
board = [[0]*N for _ in range(N)]

def is_attack(i, j):
    #checking if there is a queen in row or column
    for k in range(0,N):
        if board[i][k]==1 or board[k][j]==1:
            return True
    #checking diagonals
    for k in range(0,N):
        for l in range(0,N):
            if (k+l==i+j) or (k-l==i-j):
                if board[k][l]==1:
                    return True
    return False

def N_queen(n):
    #n is 0 -> solution found
    if n==0:
        return True
    for i in range(0,N):
        for j in range(0,N):
            if (not(is_attack(i,j))) and (board[i][j]!=1):
                board[i][j] = 1
                if N_queen(n-1)==True:
                    return True
                board[i][j] = 0

    return False

N_queen(N)

input = board
print("Output : \n")
print(" -------------------------------------------------")
for i in range(0,8):
  print(" | ",input[i][0] , " | ",input[i][1] , " | ",input[i][2] , " | ",input[i][3] , " | ",input[i][4] , " | ",input[i][5] , " | ",input[i][6] , " | ",input[i][7] ," | " )
  print(" -------------------------------------------------")

print("\n")

Output : 

 -------------------------------------------------
 |  1  |  0  |  0  |  0  |  0  |  0  |  0  |  0  | 
 -------------------------------------------------
 |  0  |  0  |  0  |  0  |  1  |  0  |  0  |  0  | 
 -------------------------------------------------
 |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  1  | 
 -------------------------------------------------
 |  0  |  0  |  0  |  0  |  0  |  1  |  0  |  0  | 
 -------------------------------------------------
 |  0  |  0  |  1  |  0  |  0  |  0  |  0  |  0  | 
 -------------------------------------------------
 |  0  |  0  |  0  |  0  |  0  |  0  |  1  |  0  | 
 -------------------------------------------------
 |  0  |  1  |  0  |  0  |  0  |  0  |  0  |  0  | 
 -------------------------------------------------
 |  0  |  0  |  0  |  1  |  0  |  0  |  0  |  0  | 
 -------------------------------------------------




**Graph Coloring:**

In [ ]:
# here our graph has 4 nodes. So we use 4*4 matrix.
# Each element of the matrix deteremine whether there is an edge between corresponding nodes or not
# 1 means edge exists, 0 means edge does not exist

def isSafe(graph, color):
	for i in range(4):
		for j in range(i + 1, 4):
			if (graph[i][j] and color[j] == color[i]):
				return False
	return True


def graphColoring(graph, m, i, color):
	if (i == 4):
		if (isSafe(graph, color)):
			printSolution(color)
			return True
		return False

	for j in range(1, m + 1):
		color[i] = j
		if (graphColoring(graph, m, i + 1, color)):
			return True
		color[i] = 0
	return False


def printSolution(color):
  print("Solution exists: \n")
  for i in range(0,4):
    if color[i] == 1:
      print("Node ",i+1,": ","Red")
    elif color[i] == 2:
      print("Node ",i+1,": ","Green")
    elif color[i] == 3:
      print("Node ",i+1,": ","Blue")
	



graph = [
  [ 0, 1, 1, 0 ],
  [ 1, 0, 1, 0 ],
  [ 1, 1, 0, 1 ],
  [ 0, 0, 1, 0 ],
]
m = 3 
color = [0 for i in range(4)]

if (not graphColoring(graph, m, 0, color)):
  print ("Solution not exist")

  




Solution exists: 

Node  1 :  Red
Node  2 :  Green
Node  3 :  Blue
Node  4 :  Red


**8 Puzzle:**

In [ ]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

In [ ]:
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):

    def __init__(self, initial=None, goal=None, **kwds): 
      self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        
      raise NotImplementedError

    def result(self, state, action):
      raise NotImplementedError

    def is_goal(self, state):        
      return state == self.goal

    def action_cost(self, s, a, s1): 
      return 1

    def h(self, node):               
      return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
      self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): 
      return '<{}>'.format(self.state)

    def __len__(self): 
      return 0 if self.parent is None else (1 + len(self.parent))

    def __lt__(self, other): 
      return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf)  #  algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf)  #  iterative deepening search was cut off.
    
    
def expand(problem, node):
    s = node.state
    for action in problem.actions(s):
      s1 = problem.result(s, action)
      cost = node.path_cost + problem.action_cost(s, action, s1)
      yield Node(s1, node, action, cost)
        

def path_actions(node):
    if node.parent is None:
      return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    if node in (cutoff, failure, None): 
      return []
    return path_states(node.parent) + [node.state]

In [ ]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
  def __init__(self, items=(), key=lambda x: x): 
    self.key = key
    self.items = []
    for item in items:
      self.add(item)
        
  def add(self, item):
    pair = (self.key(item), item)
    heapq.heappush(self.items, pair)

  def pop(self):
    return heapq.heappop(self.items)[1]
  
  def top(self): 
    return self.items[0][1]

  def __len__(self): 
    return len(self.items)

In [ ]:
def g(n): return n.path_cost

def best_first_search(problem, f):
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def astar_search(problem, h=None):
    #we find minimum f(n) = g(n) + h(n).
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


In [ ]:
class EightPuzzle(Problem):

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
      assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
      self.initial, self.goal = initial, goal
    
    def actions(self, state):
      moves = ((1, 3),    (0, 2, 4),    (1, 5),
                (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                (3, 7),    (4, 6, 8),    (7, 5))
      blank = state.index(0)
      return moves[blank]
    
    def result(self, state, action):
      s = list(state)
      blank = state.index(0)
      s[action], s[blank] = s[blank], s[action]
      return tuple(s)
    
    def h1(self, node):
      return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
      X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
      Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
      return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                  for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h(self, node): 
      X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
      Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
      return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                  for (s, g) in zip(node.state, self.goal) if s != 0)
    
    
def hamming_distance(A, B):
  return sum(a != b for a, b in zip(A, B))
    

def inversions(board):
  return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
  return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
  empty = '.'
  off = '#'
  def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
    if board is not None:
      self.update(board)
      self.width, self.height = (board.width, board.height) 
    else:
      self.width, self.height = (width, height)
    self.to_move = to_move

  def __missing__(self, key):
    x, y = key
    if x < 0 or x >= self.width or y < 0 or y >= self.height:
      return self.off
    else:
      return self.empty
      
  def __repr__(self):
    def row(y): return ' '.join(self[x, y] for x in range(self.width))
    return '\n'.join(row(y) for y in range(self.height))
          
  def __hash__(self): 
    return hash(tuple(sorted(self.items()))) + hash(self.to_move)

In [ ]:
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
for s in path_states(astar_search(e1)):
    print(board8(s))

1 4 2
_ 7 5
3 6 8

1 4 2
3 7 5
_ 6 8

1 4 2
3 7 5
6 _ 8

1 4 2
3 _ 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8

